In [ ]:
import configparser
import os

In [ ]:
from obspy.clients.filesystem.sds import Client
from obspy.clients.fdsn import RoutingClient
from obspy.core import UTCDateTime as UTC

In [ ]:

default_settings = dict(network = '*',
    station = '*',
    location = '',
    channel = '*',
    overlap = 60,
    amplitude_frequencies = (4,14),
    nperseg = 2048,
    winlen_in_s = 3600,
    proclen = 24*3600,
    sds_root = os.path.abspath('.'),
    inventory_routing_type = "eida-routing")


class RawDataProcessor():
    def __init__(self, **kwargs):
        # Set the intended attributes from default_settings
        for k, v in default_settings.items():
            self.__setattr__(k, v)
        
        # Override attributes from configfile if given
        try:
            v = kwargs.pop('configfile')
            print('configfile', v)
            self.from_config(v)
            self.configfile = v
        except KeyError:
            pass
            
        # Override attributes directly given as argument
        #for k, v in kwargs.items():
        #    try:
        #        self.__getattribute__(k)
        #    except AttributeError:
        #        raise AttributeError("Unknown keyword {}.".format(k))
        #    self.__setattr__(k, v)
        self._check_set_attr(kwargs)
        
        self.sdsclient = Client(self.sds_root)
        self.invclient = RoutingClient(self.inventory_routing_type)
        self.processed_codes = set()


    def from_config(self, configfile):

        if not os.path.isfile(configfile):
            raise FileNotFoundError("No config-file %s " % 
                    configfile)

        config = configparser.ConfigParser()
        config.read(configfile)
        
        sec = "PROCESSING PARAMETERS"
        
        # Get strings
        str_kws = [k for k, v in default_settings.items() 
           if isinstance(v, str)]
        
        for k in str_kws:
            v = config[sec].get(k) 
            self.__setattr__(k, v)
        
        # Get number parameters (we expect only ints)
        num_kws = [k for k,v in default_settings.items()
                   if any([isinstance(v, int), 
                          isinstance(v, float)])]
        for k in num_kws:
            v = config[sec].getint(k)
            self.__setattr__(k, v)
        
        # Get tuples
        k = 'amplitude_frequencies'
        v = [float(i) for i in 
             config[sec][k].split(',')]
        self.__setattr__(k, v)
        
        sec = "SDS_CLIENT_DICT"
        sds_client_dict = {}
        for k in ['sds_type', 'format']:
            v = config[sec].get(k)
            if v is not None:
                sds_client_dict[k] = v
        
        for k in ["fileborder_samples", "fileborder_seconds"]:
            v = config[sec].getint(k)
            if v is not None:
                sds_client_dict[k] = v
        self.__setattr__('sds_client_dict', sds_client_dict)
        return config
        
        
    def show_params(self):
        print(self.__dict__)
        
    
    def _check_set_attr(self, kwargs):
        # Override attributes directly given as argument
        for k, v in kwargs.items():
            try:
                self.__getattribute__(k)
            except AttributeError:
                raise AttributeError("Unknown keyword {}.".format(k))
            self.__setattr__(k, v)


In [ ]:
proc = RawDataProcessor()

proc.show_params()

In [ ]:
config = proc.from_config('config.ini')

In [ ]:
proc = RawDataProcessor(configfile='config.ini')

In [ ]:
proc.show_params()

In [ ]:
proc = RawDataProcessor(configfile='config.ini', 
                        network="KL")

In [ ]:
proc.show_params()

In [ ]:
bla = dict(h=1, n=10)

In [ ]:
defaults = dict(network = '*',
    station = '*',
    location = '',
    channel = '*',
    overlap = 60,
    amplitude_frequencies = (4,14),
    #fmin = 4,
    #fmax = 14,
    nperseg = 2048,
    winlen_in_s = 3600,
    proclen = 24*3600,
    sds_root = os.path.abspath('.'),
    inventory_routing_type = "eida-routing",
    sds_client_dict = {})

In [ ]:
config_from_dict = configparser.ConfigParser()

sec = "PROCESSING PARAMETERS"
config_from_dict[sec] = {}
for k, v in defaults.items():
    print(k, v)
    if isinstance(v, str):
        v = v
    elif isinstance(v, int) or isinstance(v, float):
        v = str(v)
    elif isinstance(v, list) or isinstance(v, tuple):
        v = ','.join([str(vi) for vi in v])
    elif isinstance(v, dict):
        config_from_dict.read_dict({k.upper(): v})
        continue
    config_from_dict.set(sec, k, v)
    

with open('config.ini', 'w') as fp:
    config_from_dict.write(fp)

In [ ]:
cin = configparser.ConfigParser()
cin.read('config.ini')

In [ ]:
str_kws = [k for k, v in default_settings.items() 
           if isinstance(v, str)]

In [ ]:
sec = "PROCESSING PARAMETERS"

for k,v in cin[sec].items():
    print(k, v)

In [ ]:
for k in str_kws:
    #print(k)
    v = cin[sec].get(k) 
    #self.__setattr__(k, v)
    
    print(k, v)

In [ ]:
sec = "SDS_CLIENT_DICT"

In [ ]:
sds_client_dict = {}
for k in ['sds_type', 'format']:
    v = cin[sec].get(k)
    if v is not None:
        sds_client_dict[k] = v


In [ ]:
sds_client_dict

In [ ]:
sdsclient = Client('.', **sds_client_dict)

In [ ]:
sdsclient.__dict__

In [ ]:
for sec, params in cin.items():
    print(sec, params)
    for k, v in params.items():
        print(k, v)
        

In [ ]:
params['amplitude_frequencies']

In [ ]:
defaults.update({'configfile':'config.ini'})

In [ ]:
defaults

In [ ]:
network = '*'
station = '*'
channel = '*'

overlap = 60 #3600
fmin, fmax = (4, 14)
nperseg = 2048
winlen_in_s = 3600
proclen = 24*3600

sds_root = os.path.abspath('../sample_sds/')
inventory_routing_type = "eida-routing"

In [ ]:
config = configparser.ConfigParser()
sec = "NETWORK"
config[sec] = {}
config[sec]["network"] = network
config[sec]["station"] = station
config[sec]["channel"] = channel

sec = "PROCESSING"
config[sec] = {}
config[sec]["overlap"] = str(overlap)
config[sec]["fmin"] = str(fmin)
config[sec]["fmax"] = str(fmax)
config[sec]["nperseg"] = str(nperseg)
config[sec]["winlen_in_s"] = str(winlen_in_s)
config[sec]["proclen"] = str(proclen)
config[sec]["sds_root"] = sds_root
config[sec]["inventory_routing_type"] = inventory_routing_type

In [ ]:
with open('config.ini', 'w') as fp:
    config.write(fp)

In [ ]:
sdsclient = Client(sds_root)
invclient = RoutingClient(inventory_routing_type)

In [ ]:
from importlib import reload

In [ ]:
from data_quality_control import processing

In [ ]:
reload(processing)

In [ ]:
c = processing.get_default_settings()